<a href="https://www.kaggle.com/code/gazinator/agedetectionfinal?scriptVersionId=129016259" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from PIL import Image
from keras import layers

In [ ]:
curdir = "../input/utkface-new/UTKFace"
dataset_filepaths = os.listdir(curdir)
tf.random.set_seed(42)
np.random.seed(42)
np.random.shuffle(dataset_filepaths)

In [ ]:
BytesList = tf.train.BytesList
FloatList = tf.train.FloatList
Int64List = tf.train.Int64List
Feature = tf.train.Feature
Features = tf.train.Features
Example = tf.train.Example

In [ ]:
def create_example(folder_path, filepath):
    full_path = os.path.join(folder_path, filepath)
    image = tf.io.serialize_tensor(np.array(Image.open(full_path)))
    split = filepath.split("_")
    age = int(split[0])
    gender = int(split[1])
    
    example = Example(
        features=Features(
            feature={
                "image": Feature(bytes_list=BytesList(value=[image.numpy()])),
                "age": Feature(int64_list=Int64List(value=[age])),
            }
        )
    )
    return example

In [ ]:
def create_tf_record(set_, filename):
    with tf.io.TFRecordWriter("%s.tfrecord" %filename) as f:
        for filepath in set_:
            example = create_example(curdir, filepath)
            f.write(example.SerializeToString())

In [ ]:
create_tf_record(dataset_filepaths[:18968], "train_data")
create_tf_record(dataset_filepaths[18968:], "valid_data")

In [ ]:
@tf.function
def preprocess(tfrecord):
    feature_descriptions = {
        "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "age": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
    }
    example = tf.io.parse_single_example(tfrecord, feature_descriptions)
    image = tf.io.parse_tensor(example["image"], out_type=tf.uint8)
    image = tf.reshape(image, shape=[200, 200, 3])
    image = tf.image.resize(image, [224,224]) 
    image = keras.applications.xception.preprocess_input(image)
    return image, example["age"]

def utkface_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None,
                  n_parse_threads=5, batch_size=32, cache=True):
    dataset = tf.data.TFRecordDataset(filepaths,
                                      num_parallel_reads=n_read_threads)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [ ]:
train_set = utkface_dataset("./train_data.tfrecord")
valid_set = utkface_dataset("./valid_data.tfrecord")

In [ ]:
inputs = keras.layers.Input((224,224,3))

conv_1 = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu') (inputs)
maxp_1 = keras.layers.MaxPooling2D(pool_size=(2, 2)) (conv_1)
conv_2 = keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu') (maxp_1)
maxp_2 = keras.layers.MaxPooling2D(pool_size=(2, 2)) (conv_2)
conv_3 = keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu') (maxp_2)
maxp_3 = keras.layers.MaxPooling2D(pool_size=(2, 2)) (conv_3)
conv_4 = keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu') (maxp_3)
maxp_4 = keras.layers.MaxPooling2D(pool_size=(2, 2)) (conv_4)

flatten = keras.layers.Flatten() (maxp_4)

dense_2 = keras.layers.Dense(256, activation='relu') (flatten)

dropout_2 = keras.layers.Dropout(0.3) (dense_2)

output_2 = keras.layers.Dense(1, activation='relu', name='age_out') (dropout_2)

model = tf.keras.Model(inputs=[inputs], outputs=output_2)

model.compile(loss='mae', optimizer='nadam')

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_model.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,
                                                  restore_best_weights=True)
history = model.fit(train_set,
                    validation_data=valid_set,
                    epochs=50,
                    callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
model.save('my_model.h5')